In [ ]:
!pip install "tensorflow>=1.7.0"
!pip install tensorflowjs
!pip install jax==0.4.21
!pip install jaxlib==0.4.21
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import json
import numpy as np
import tensorflowjs as tfjs
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
def split_segments(image):
    x, y, w, h = cv2.boundingRect(image)
    segments = []

    def recursive_split(x, y, w, h):
        if w > h-5:
            mid = x + w // 2
            recursive_split(x, y, mid - x, h)
            recursive_split(mid, y, x + w - mid, h)
        else:
            segments.append((x, y, w, h))

    recursive_split(x, y, w, h)
    return segments

def pad_to_center(image, target_size=(30, 30)):
        h, w = image.shape[:2]
        dh, dw = target_size[0] - h, target_size[1] - w
        top, left = dh // 2, dw // 2
        bottom, right = dh - top, dw - left

        padding = ((top, bottom), (left, right)) + ((0, 0),) * (image.ndim - 2)
        return np.pad(image, padding, 'constant')

In [ ]:
l = '928153'
files = os.listdir(l)
for i, file in enumerate(files):
    image = cv2.imread(os.path.join(l, f'{i}.png'), 0)
    _, image = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda cnt: cv2.boundingRect(cnt)[0])

    digit_images = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        if w >= 6 and h >= 6:
            if w > h:
                segments = split_segments(cnt)
            else:
                segments = [(x, y, w, h)]

            for sx, sy, sw, sh in segments:
                digit = image[sy:sy+sh, sx:sx+sw]
                digit_images.append(digit)

    for j, digit in enumerate(digit_images):
        digit = pad_to_center(digit)

        if not os.path.exists(l[j]):
            os.makedirs(l[j])

        cv2.imwrite(os.path.join(l[j], f'{i}.png'), digit)

In [ ]:
!zip -r data.zip ./

In [ ]:
!unzip data.zip

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(30, 30, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import random

def add_noise(img):
    rotation_degree = random.randint(-45, 45)
    rows, cols, _ = img.shape
    M = cv2.getRotationMatrix2D((cols/2, rows/2), rotation_degree, 1)
    img = cv2.warpAffine(img, M, (cols, rows))

    number_of_lines = random.randint(1, 5)
    for _ in range(number_of_lines):
        x1, y1 = random.randint(0, cols), random.randint(0, rows)
        x2, y2 = random.randint(0, cols), random.randint(0, rows)
        img = cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)

    noise_ratio = 0.3
    noise = np.random.choice([0, 1], size=img.shape, p=[1 - noise_ratio, noise_ratio])
    mask = np.random.choice([0, 255], size=img.shape, p=[1 - noise_ratio, noise_ratio])
    img = np.where(noise == 1, mask, img)
    return img

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1,
    preprocessing_function=add_noise
    )

batch_size = 32
train_generator = datagen.flow_from_directory(
    'data',
    target_size=(30, 30),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    'data',
    target_size=(30, 30),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 897 images belonging to 10 classes.
Found 95 images belonging to 10 classes.


In [ ]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import itertools

test_datagen = ImageDataGenerator(rescale=1.0/255, preprocessing_function=add_noise)

test_generator = test_datagen.flow_from_directory(
    'data',
    target_size=(30, 30),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(10,10))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)

fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

report = classification_report(test_generator.classes, predicted_classes)
print(report)

In [ ]:
model.save('model.h5')
tfjs.converters.save_keras_model(model, "tfjs.h5")

In [ ]:
import requests
from PIL import Image
from keras.preprocessing import image
import io

def loadimg(url):
    response = requests.get(url)
    image_data = response.content
    with open('test.png', 'wb') as f:
        f.write(image_data)

def procimg(img):
    image = cv2.imread(img, 0)
    _, image = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda cnt: cv2.boundingRect(cnt)[0])

    digit_images = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        if w >= 6 and h >= 6:
            if w > h:
                segments = split_segments(cnt)
            else:
                segments = [(x, y, w, h)]

            for sx, sy, sw, sh in segments:
                digit = image[sy:sy+sh, sx:sx+sw]
                digit_images.append(digit)
    return digit_images


from keras.models import load_model
model = load_model('model.h5')
loadimg('https://www.ccxp.nthu.edu.tw/ccxp/INQUIRE/auth_img.php?pwdstr=20240111-789039411757')
digits = procimg('test.png')

ans = []
for i, digit in enumerate(digits):
    digit = pad_to_center(digit)
    img = cv2.cvtColor(digit, cv2.COLOR_BGR2RGB)
    #plt.imshow(img)
    #plt.show()
    img = cv2.resize(img, (30, 30))
    img = image.img_to_array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    predictions = model.predict(img)
    ans.append(np.argmax(predictions))

print(''.join(map(str, ans)))

1/1 [==============================] - 0s 30ms/step
520205


In [ ]:
import tensorflow as tf
model = load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open ("model.tflite" , "wb") .write(tflite_model)